In [2]:
import bz2
import numpy as np
from tqdm import tqdm_notebook as tqdm
import gzip
from heapq import heappushpop
from joblib import Parallel, delayed
import time

In [3]:
import numpy as np
import os

classes = []

for root, dirs, files in os.walk("."):  
    for filename in files:
        classes.append(filename)

In [11]:
from re import compile

rex = compile('n[0-9]+')
classes = [x for x in classes if rex.match(x)]

In [12]:
x_trn = []
y_trn = []
for c in tqdm(classes):
    x_trn.append(np.load(c))
    y_trn += [c] * len(x_trn[-1])

In [13]:
x_trn = np.vstack(x_trn)
print(len(x_trn))
print(len(y_trn))

997659
997659


In [14]:
from sklearn.utils import shuffle
x_trn, y_trn = shuffle(x_trn, y_trn, random_state=0)

In [15]:
x_trn = np.reshape(x_trn, (-1, 2048))
x_tst, y_tst = x_trn[:100], y_trn[:100]
x_val, y_val = x_trn[100:1100], y_trn[100:1100]
x_trn, y_trn = x_trn[1100:], y_trn[1100:]

In [35]:
def get_true_KNN(x_trn,x_tst):
    N = x_trn.shape[0]
    N_tst = x_tst.shape[0]
    x_tst_knn_gt = np.zeros((N_tst, N))
    for i_tst in tqdm(range(N_tst)):
        dist_gt = np.zeros(N)
        for i_trn in range(N):
            dist_gt[i_trn] = np.linalg.norm(x_trn[i_trn,:] - x_tst[i_tst, :], 2)
#             if i_trn % 100000 == 0:
#                 print(i_trn)
        x_tst_knn_gt[i_tst, :] = np.argsort(dist_gt)
        # x_sort_part = np.argpartition(dist_gt,K)[:K]
        # x_tst_knn_gt[i_tst, :] = x_sort_part[np.argsort(dist_gt[x_sort_part])]
#         if i_tst % 100 == 0:
#             print(i_tst)

    return x_tst_knn_gt.astype(int)

start = time.time()
x_tst_knn_gt = get_true_KNN(x_trn, x_tst)
end1 = time.time() - start
print(end1)

start = time.time()
x_val_knn_gt = get_true_KNN(x_trn, x_val)
val_end1 = time.time() - start
print(val_end1)

702.9819550514221


6849.323750257492


In [17]:
def compute_single_unweighted_knn_class_shapley(x_trn,y_trn,x_tst_knn_gt,y_tst,K):
    N = x_trn.shape[0]
    N_tst = x_tst_knn_gt.shape[0]
    sp_gt = np.zeros((N_tst,N))
    for j in tqdm(range(N_tst)):
        # try:
        sp_gt[j,x_tst_knn_gt[j,-1]] = (y_trn[x_tst_knn_gt[j,-1]] == y_tst[j])/N
        # except:
        #     pdb.set_trace()
        for i in np.arange(N-2,-1,-1):
            sp_gt[j,x_tst_knn_gt[j,i]] = sp_gt[j,x_tst_knn_gt[j,i+1]] +\
                           (int(y_trn[x_tst_knn_gt[j,i]] == y_tst[j]) -
                            int(y_trn[x_tst_knn_gt[j,i+1]] == y_tst[j]))/K* min([K,i+1])/(i+1)
            # if j == 404 and i == 0:
            #     pdb.set_trace()
    return sp_gt

In [36]:
start = time.time()
sp_gt = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_tst_knn_gt, y_tst, 1)
end2 = time.time() - start

start = time.time()
val_sp_gt = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_val_knn_gt, y_val, 1)
val_end2 = time.time() - start

In [41]:
K = 2
start2 = time.time()
sp_gt2 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_tst_knn_gt, y_tst, K)
end2 = time.time() - start2

In [42]:
K = 5
start5 = time.time()
sp_gt5 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_tst_knn_gt, y_tst, 5)
end5 = time.time() - start5

In [25]:
K = 10
dist_rand = np.load('eps0.1/dist_rand.npy')
contrast = np.load('eps0.1/contrast.npy')
dist_knn = np.load('eps0.1/dist_knn.npy')

dist_rand = np.mean(dist_rand, axis=0)
contrast = np.mean(contrast, axis=0)[K-1]
dist_knn = np.mean(dist_knn, axis=0)[K-1]

In [28]:
search_range = np.arange(1e-3, 10, 1e-3)
from scipy.stats import norm


def f_h(x, r):
    y = 1-2*norm.cdf(-r/x) - 2/(np.sqrt(2*np.pi)*r/x)*(1-np.exp(-(r**2/(2*(x**2)))))
    return y

def g_unnormalize(dist_rand,dist_knn,r):
    y = np.log(f_h(dist_knn,r))/np.log(f_h(dist_rand,r))
    return y

def g_normalize(contrast,r):
    y = np.log(f_h(1/contrast,r))/np.log(f_h(1,r))
    return y

def find_best_r_normalize(search_range,contrast):
    y = g_normalize(contrast,search_range)
    min_ind = np.argmin(y)
    return search_range[min_ind]

def find_best_r_unnormalize(search_range,dist_rand,dist_knn):
    y = g_unnormalize(dist_rand,dist_knn,search_range)
    min_ind = np.argmin(y)
    return search_range[min_ind]

r_vec_normalize = find_best_r_normalize(search_range, contrast)
g_vec = g_normalize(contrast, r_vec_normalize)

In [29]:
t = r_vec_normalize
n_trn = len(x_trn)
n_hash_table = 10 # 如果 estimation error 高于 0.01
n_hash_bit = int(np.ceil(np.log(n_trn)*0.5/ np.log(1 / f_h(1, t))))
print(n_hash_bit)

17


In [30]:
import numpy as np
import pdb


def lsh_function(t,x,w,b):
    # x is 1-d array
    h = np.floor((np.dot(w,x)+b)/t).astype(int)
    return h


class LSH:
    def __init__(self,n_hash_bit,n_hash_table,x_trn,y_trn,t=0.1):
        self.n_hash_bit = n_hash_bit
        self.n_hash_table = n_hash_table
        self.t = t # width of projections
        self.x_trn = x_trn
        self.y_trn = y_trn
        self.N, self.dim = x_trn.shape
        # draw w from a normal distribution (2-stable)
        self.w = np.random.normal(0, 1, (n_hash_table, n_hash_bit, self.dim))
        # draw b from U[0,t]
        self.b = np.random.uniform(0, self.t, (n_hash_table, n_hash_bit))
        self.x_trn_hash = [dict() for i in range(n_hash_table)]
        for i in tqdm(range(self.N)):
            hash_code_all = lsh_function(self.t, x_trn[i] / dist_rand, self.w, self.b)
            for l in range(n_hash_table):
                hash_code_trn = '.'.join(map(str, hash_code_all[l, :]))
                if hash_code_trn in self.x_trn_hash[l].keys():
                    self.x_trn_hash[l][hash_code_trn].append(i)
                else:
                    self.x_trn_hash[l][hash_code_trn] = [i]
#             if i % 1000 == 0:
#                 print('build hash %s'%i)

    def get_approx_KNN(self,x_tst,K):
        N_tst = x_tst.shape[0]
        x_tst_knn = np.ones((N_tst, K)) * (-1)
        nns_len = np.zeros(N_tst)
        for i_tst in tqdm(range(N_tst)):
            nns = []
            for l in range(self.n_hash_table):
                hash_code_int = lsh_function(self.t, x_tst[i_tst] / dist_rand, self.w[l, :, :], self.b[l, :])
                hash_code_test = '.'.join(map(str, hash_code_int))
                if hash_code_test in self.x_trn_hash[l].keys():
                    nns += self.x_trn_hash[l][hash_code_test]
            nns = np.unique(nns)
            num_collide_elements = len(nns)
            if len(nns) > 0:
                dist = [np.linalg.norm(self.x_trn[i] / dist_rand - x_tst[i_tst] / dist_rand, 2) for i in nns]
                dist_min_ind = nns[np.argsort(dist)]
                if num_collide_elements < K:
                    x_tst_knn[i_tst, :num_collide_elements] = dist_min_ind[:num_collide_elements]
                else:
                    x_tst_knn[i_tst, :] = dist_min_ind[:K]
            # pdb.set_trace()
            nns_len[i_tst] = len(nns)
            if i_tst % 100 == 0:
                print('get approximate knn %s'%i_tst)
        return x_tst_knn.astype(int),nns_len


    def compute_approx_shapley(self,x_tst_knn,y_tst,K):
        N_tst,K_star = x_tst_knn.shape
        # flag_sufficient = (x_tst_knn[:,-1]>=0)
        sp_approx = np.zeros((N_tst,self.N))
        for j in tqdm(range(N_tst)):
            non_nan_index = np.where(x_tst_knn[j,:]>=0)[0]
            if len(non_nan_index)== 0:
                continue
            K_tot = non_nan_index[-1]
            if K_tot == self.N:
                sp_approx[j, x_tst_knn[j, self.N - 1]] = equal(self.y_trn[x_tst_knn[j, self.N - 1]], y_tst[j]) / self.N
            for i in np.arange(K_tot - 1, -1, -1):
                sp_approx[j, x_tst_knn[j, i]] = sp_approx[j, x_tst_knn[j, i+1]] + (
                        equal(self.y_trn[x_tst_knn[j, i]], y_tst[j]) - equal(
                    self.y_trn[x_tst_knn[j, i + 1]], y_tst[j])) / K * min([K, i + 1]) / (i + 1)



        return sp_approx

In [31]:
def equal(a, b):
    return a == b
    
def fine_tune(n_hash_table=10, alpha=0.5, file=False):
    t = r_vec_normalize
    n_trn = len(x_trn)
    n_hash_bit = int(np.ceil(np.log(n_trn)*alpha/ np.log(1 / f_h(1, t))))
    if file is True:
        print(n_hash_bit, file=open('eps0.1/log.txt', 'a'))
    else:
        print(n_hash_bit)
    
    start = time.time()
    lsh = LSH(n_hash_bit=n_hash_bit, n_hash_table=n_hash_table, x_trn=x_trn, y_trn=y_trn, t=t)
    runtime_build_hash = time.time() - start
    if file is True:
        print(runtime_build_hash, file=open('eps0.1/log.txt', 'a'))
    else:
        print(runtime_build_hash)
    
    K_star = 10
    start = time.time()
    x_val_knn_approx, nns_vec = lsh.get_approx_KNN(x_val, K_star)
    runtime_query = time.time() - start
    if file is True:
        print(runtime_query, file=open('eps0.1/log.txt', 'a'))
    else:
        print(runtime_query)
    
    K = 1
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_val_knn_approx, y_val, K)
    runtime_approx_value = time.time() - start
    if file is True:
        print('it takes %s to get appox knn value' % runtime_approx_value, file=open('eps0.1/log.txt', 'a'))
    else:
        print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(val_sp_gt - sp_approx,ord=np.inf, axis=1)
    if file is True:
        print('max error %s'% np.percentile(sp_err_inf_val,90), file=open('eps0.1/log.txt', 'a'))
    else:
        print('max error %s'% np.percentile(sp_err_inf_val,90))
    return lsh

In [32]:
def fine_tune_test(n_hash_table=10, lsh=None, file=False):
    if not lsh:
        t = r_vec_normalize
        n_trn = len(x_trn)
        n_hash_bit = int(np.ceil(np.log(n_trn)*0.3/ np.log(1 / f_h(1, t))))
        print(n_hash_bit)
    
        start = time.time()
        lsh = LSH(n_hash_bit=n_hash_bit, n_hash_table=n_hash_table, x_trn=x_trn_norm, y_trn=y_trn, t=t)
        runtime_build_hash = time.time() - start
        print(runtime_build_hash)
    
    K_star = 10
    start = time.time()
    x_tst_knn_approx, nns_vec = lsh.get_approx_KNN(x_tst, K_star)
    runtime_query = time.time() - start
    if file is True:
        print(runtime_query, file=open('eps0.1/log.txt', 'a'))
    else:
        print(runtime_query)
    
    K = 1
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_tst_knn_approx, y_tst, K)
    runtime_approx_value = time.time() - start
    if file is True:
        print('it takes %s to get appox knn value' % runtime_approx_value, file=open('eps0.1/log.txt', 'a'))
    else:
        print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(sp_gt - sp_approx,ord=np.inf, axis=1)
    if file is True:
        print('max error %s'% np.percentile(sp_err_inf_val,90), file=open('eps0.1/log.txt', 'a'))
    else:
        print('max error %s'% np.percentile(sp_err_inf_val,90))
    return sp_approx, nns_vec

In [33]:
lsh_195_05 = fine_tune(195, 0.5, file=True)
sp_approx_195_05, nns_vec_195_05 = fine_tune_test(195, lsh_195_05, file=True)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900


NameError: name 'val_sp_gt' is not defined

In [37]:
lsh_195_05 = fine_tune(195, 0.5, file=True)
sp_approx_195_05, nns_vec_195_05 = fine_tune_test(195, lsh_195_05, file=True)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900


get approximate knn 0


```
17
6120.1388120651245
2757.237851858139
it takes 0.15042829513549805 to get appox knn value
max error 0.0954959090259068
251.7484724521637
it takes 0.025987863540649414 to get appox knn value
max error 0.1114729758955185
```

In [38]:
lsh_200_05 = fine_tune(200, 0.5, file=True)
sp_approx_200_05, nns_vec_200_05 = fine_tune_test(200, lsh_200_05, file=True)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900


get approximate knn 0


```
17
7756.053581237793
2403.7759668827057
it takes 0.09811997413635254 to get appox knn value
max error 0.0900423957364133
255.54018378257751
it takes 0.06232953071594238 to get appox knn value
max error 0.09245485250425375
```

In [39]:
lsh_190_05 = fine_tune(190, 0.5, file=True)
sp_approx_190_05, nns_vec_190_05 = fine_tune_test(190, lsh_190_05, file=True)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900


get approximate knn 0


```
17
6549.824974775314
2130.7291095256805
it takes 0.10925650596618652 to get appox knn value
max error 0.1073674237593991
226.62567830085754
it takes 0.026142358779907227 to get appox knn value
max error 0.13142807051881394
```

In [40]:
lsh_196_05 = fine_tune(196, 0.5, file=True)
sp_approx_196_05, nns_vec_196_05 = fine_tune_test(196, lsh_196_05, file=True)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900


get approximate knn 0


```
17
6838.125294446945
2439.2873623371124
it takes 0.10068678855895996 to get appox knn value
max error 0.0965434712180332
252.69935274124146
it takes 0.026296377182006836 to get appox knn value
max error 0.10028850501345016
```

In [44]:
lsh = lsh_200_05
def test_error(K, sp_gt):
    K_star = 10
    start = time.time()
    x_tst_knn_approx, nns_vec = lsh.get_approx_KNN(x_tst, K_star)
    runtime_query = time.time() - start
    print(runtime_query)
    
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_tst_knn_approx, y_tst, K)
    runtime_approx_value = time.time() - start
    print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(sp_gt - sp_approx,ord=np.inf, axis=1)
    print('max error %s'% np.percentile(sp_err_inf_val,90))
    return sp_approx

In [45]:
sp_gt2_approx = test_error(2, sp_gt2)

get approximate knn 0
328.5784842967987


it takes 0.22670292854309082 to get appox knn value
max error 0.09245485250425375


In [46]:
sp_gt5_approx = test_error(5, sp_gt5)

get approximate knn 0
324.9600188732147


it takes 0.028543949127197266 to get appox knn value
max error 0.08566220199647492


In [69]:
def val_error(K, val_sp_gt):
    K_star = 10
    start = time.time()
    x_val_knn_approx, nns_vec = lsh.get_approx_KNN(x_val, K_star)
    runtime_query = time.time() - start
    print(runtime_query)
    
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_val_knn_approx, y_val, K)
    runtime_approx_value = time.time() - start
    print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(val_sp_gt - sp_approx,ord=np.inf, axis=1)
    print('max error %s'% np.percentile(sp_err_inf_val,90))
    return sp_approx

In [50]:
lsh = lsh_195_05
sp_gt2_approx2 = val_error(2, sp_gt2)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900
3063.849513530731


it takes 0.11894392967224121 to get appox knn value
max error 0.49940666531545885


In [51]:
sp_gt5_approx2 = val_error(5, sp_gt5)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900
2916.8877487182617


it takes 0.11890006065368652 to get appox knn value
max error 0.6848752968987817


In [ ]:
tst_sp_gt2_approx2 = test_error(2, sp_gt2)
tst_sp_gt5_approx2 = test_error(5, sp_gt5)

In [ ]:
lsh = lsh_200_05
val_sp_gt2_approx = val_error(2, sp_gt2)

In [ ]:
val_sp_gt5_approx = val_error(5, sp_gt5)

In [53]:
tst_sp_gt2_approx2 = test_error(2, sp_gt2)

get approximate knn 0
297.74391746520996


it takes 0.03178548812866211 to get appox knn value
max error 0.1114729758955185


In [54]:
tst_sp_gt5_approx2 = test_error(5, sp_gt5)

get approximate knn 0
326.85515546798706


it takes 0.029559850692749023 to get appox knn value
max error 0.09716649796210718


In [55]:
lsh.n_hash_table

195

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [57]:
lsh_200_05.x_trn = []
lsh_200_05.y_trn = []
np.save('eps0.1/lsh_200_05', lsh_200_05)

In [58]:
np.save('eps0.1/x_trn', x_trn)

In [59]:
np.save('eps0.1/y_trn', y_trn)

In [60]:
np.save('eps0.1/x_val', x_val)

In [61]:
np.save('eps0.1/y_trn', y_val)

In [62]:
np.save('eps0.1/x_tst', x_tst)

In [63]:
np.save('eps0.1/y_tst', y_tst)

In [ ]:
K = 2
start2 = time.time()
val_sp_gt2 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_val_knn_gt, y_val, K)
end2 = time.time() - start2
print(end2)

In [68]:
K = 5
start5 = time.time()
val_sp_gt5 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_val_knn_gt, y_val, K)
end5 = time.time() - start5
print(end5)

5546.4469838142395


In [70]:
val_sp_gt2_approx2 = val_error(2, val_sp_gt2)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900
3079.762225151062


it takes 0.1383364200592041 to get appox knn value
max error 0.09527141757441059


In [72]:
val_sp_gt5_approx2 = val_error(5, val_sp_gt5)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900
3043.701558113098


it takes 0.12779903411865234 to get appox knn value
max error 0.09053015135602935
